## Import packages used

In [ ]:
import os
os.chdir('/Users/m.egelundmuller/Documents/GitHub/midasmlpy')
import numpy as np
import pandas as pd
from datetime import datetime
from VariousJupyterScripts.date_functions import * # used to handle different frequencies of data and to create lags
from VariousJupyterScripts.sparse_group_lasso import bestmodel
from sklearn.model_selection import KFold

## Load data

Load data from excel

In [ ]:
# load data from xlsx files and create a dataframe
Predictors = pd.read_excel('midasmlpy/data_files/predictors-monthly.xlsx').to_numpy()
Target = pd.read_excel('midasmlpy/data_files/recessions-quarterly.xlsx').to_numpy()

Split data into dates and data tables

In [ ]:
# Y data and X and Y dates can also be defined as they are the same for all iterations
Y_date = Target[:,0]
Y = Target[:,1]
X_date = Predictors[:,0]
X = Predictors[:,1:]

## Transform data using functions from data_functions

Define variables ued in transformation

In [ ]:
# Lag variables
x_lags = 3
y_lags = 0
horizon = 0

# Legendre matrix
legendre_degree = 3

Call data transformation function

In [ ]:
transformed_data = data_transform(Y, Y_date, X, X_date, x_lags, y_lags, horizon, legendre_degree=3, standardize = True)

## sgLasso

Use log_sparse_four. Idea is: User inserts x_tilde, y, legendre degree

In [ ]:
# Function input
x = transformed_data['X_tilde']
y = transformed_data['Y']
group_size = legendre_degree
alsparse = 0.5
# nalam, b0, beta, activeGroup, nbeta, alam, npass, jerr = sgl(x, y, group_size, alsparse, intr = True, pmax = 366)
nlam = 100
pmax = 122
intr = True
k_folds = 5

In [ ]:
# bestmodel = bestmodel(x,y,group_size, alsparse, nlam = 100, pmax = 122, intr = True,k_folds = 5)
# bestmodel['maximized_performance']

In [ ]:
# Parameters
alsparse_values = np.linspace(0, 1, 21)  # Testing alsparse from 0 to 1 in steps of 0.1

# Dictionary to store the average maximized performances for each alsparse
performance_dict = {}

# Cross-validation process
for alsparse in alsparse_values:
    performances = []
    model_result = bestmodel(x,y,group_size, alsparse, nlam = 100, pmax = 122, intr = True,k_folds = 5)
    # Append the maximized performance of this fold
    performance_dict[alsparse] = model_result['maximized_performance']


# Determine the alsparse value with the highest average maximized performance
best_alsparse = max(performance_dict, key=performance_dict.get)
best_performance = performance_dict[best_alsparse]

In [ ]:
performance_dict